這次作業的要求在 HW02.pdf 中第 10 頁
- Simple baseline
    - You should able to pass the simple baseline using the sample code provided. --> 什麼都沒更動的時候。
- Strong baseline
    - Model architecture (layers? dimension? activation function?)
    - Training (batch size? optimizer? learning rate? epoch?)
    - Tips (batch norm? dropout? regularization?)

**這次作業在於要靠自己去嘗試調參數啦！**

資料已經處理好了，也不知道長什麼樣子。300 MB 解壓縮後很大包 1.5 GB ，載入花了 22 秒很久，但是使用 Simple baseline 的版本訓練不會太久。

擷取資料集：
Size of training set: (983945, 429)
Size of validation set: (245987, 429)

參考：
- https://www.youtube.com/watch?v=OP5HcXJg2Aw


## Model architecture 改進

- http://ielab.ie.nthu.edu.tw/108_IIE_project/3/word/108IIE_proj3_107034703_word.pdf
- https://hackmd.io/@lido2370/SJMPbNnKN

我一開始是想要用 CNN 試試看，但是 CNN 至少要求 2 dimonsion input，而且最好是正方形的輸入，但是我湊不到正方形，所以作罷。另外 CNN 要自己去算輸入與輸出層的 dimension，不會計算導致輸出有誤。

後來思考了一下，這個作業是很前期的作業，可能只要用 Fully Connected Layer 去測試就好而已，所以就是多加幾行 FC

```python
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer2048 = nn.Linear(429, 2048)
        self.layer1024 = nn.Linear(2048, 1024)
        self.layer512 = nn.Linear(1024, 512)
        self.layer256 = nn.Linear(512,256)
        self.layer128 = nn.Linear(256, 128)
        self.layer39 = nn.Linear(128, 39)
        self.out = nn.Linear(128, 39) 

        self.act_fn = nn.Sigmoid()

    def forward(self, x):
        x = self.layer2048(x)
        x = self.act_fn(x)

        x = self.layer1024(x)
        x = self.act_fn(x)

        x = self.layer512(x)
        x = self.act_fn(x)

        x = self.layer256(x)
        x = self.act_fn(x)

        x = self.layer128(x)
        x = self.act_fn(x)

        x = self.out(x)
        
        return x

```

epoch 改為 4 次，執行之後的輸出：
```
[001/004] Train Acc: 0.404591 Loss: 2.054130 | Val Acc: 0.525853 loss: 1.603278
saving model with acc 0.526
[002/004] Train Acc: 0.560343 Loss: 1.491168 | Val Acc: 0.603922 loss: 1.349655
saving model with acc 0.604
[003/004] Train Acc: 0.619581 Loss: 1.278985 | Val Acc: 0.637143 loss: 1.211290
saving model with acc 0.637
[004/004] Train Acc: 0.655806 Loss: 1.143719 | Val Acc: 0.664686 loss: 1.110771
saving model with acc 0.665
```

通常每輪（epoch） 訓練，Acc 通常會多一點點

修改 BATCH_SIZE = 32 後，執行第二次的輸出：
```
[001/004] Train Acc: 0.442119 Loss: 1.917239 | Val Acc: 0.564473 loss: 1.495950
saving model with acc 0.564
[002/004] Train Acc: 0.595558 Loss: 1.368507 | Val Acc: 0.633241 loss: 1.229036
saving model with acc 0.633
[003/004] Train Acc: 0.651652 Loss: 1.156283 | Val Acc: 0.668320 loss: 1.095194
saving model with acc 0.668
[004/004] Train Acc: 0.685776 Loss: 1.032083 | Val Acc: 0.681768 loss: 1.041329
saving model with acc 0.682
```

在其中一層加入 Dropout layer

```python
    ...
    x = self.drop1(x)

    x = self.layer128(x)
    x = self.act_fn(x)

    x = self.out(x)
        
```

輸出：
```
[001/004] Train Acc: 0.432475 Loss: 1.947828 | Val Acc: 0.555814 loss: 1.510319
saving model with acc 0.556
[002/004] Train Acc: 0.587345 Loss: 1.393966 | Val Acc: 0.630143 loss: 1.240262
saving model with acc 0.630
[003/004] Train Acc: 0.643952 Loss: 1.182250 | Val Acc: 0.667779 loss: 1.100147
saving model with acc 0.668
[004/004] Train Acc: 0.679541 Loss: 1.054692 | Val Acc: 0.682747 loss: 1.038554
saving model with acc 0.683
```

沒什麼變化。